In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
sns.set()

import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, HTML
display(HTML(r"<style>.output {display: flex; \
              align-items: center; \
              text-align: center;} \
              </style>"))

## Inferential Statistics

In [9]:
heart_prev = pd.read_csv("heart_disease.csv")
heart_prev.drop("Unnamed: 0", axis=1, inplace=True)
display(heart_prev.head())
display(heart_prev.info())

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,median_household_income,Population,Population Group,Restaurant Group,Income Group
0,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472,5.0,32273.0,22865.0,<30000,0-7,<50000
1,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705,4.0,18402.0,12350.0,<30000,0-7,<50000
2,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889,0.0,21737.0,11611.0,<30000,0-7,<50000
3,1108,23.844497,30.236068,35.381122,34.251449,35.500119,12.581968,6.943035,2.0,34064.0,26688.0,<30000,0-7,<50000
4,1109,23.329629,32.285360,37.212078,33.779183,34.154532,13.474276,6.706302,2.0,33376.0,30250.0,>=30000,0-7,<50000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4396 entries, 0 to 4395
Data columns (total 14 columns):
Zip Code                    4396 non-null int64
Smoking Prevalence          4396 non-null float64
Hypertension Prevalence     4396 non-null float64
Obesity Prevalence          4396 non-null float64
Sedentarism Prevalence      4396 non-null float64
Cholesterol Prevalence      4396 non-null float64
Diabetes Prevalence         4396 non-null float64
Heart Disease Prevalence    4396 non-null float64
Restaurant Count            4396 non-null float64
median_household_income     4396 non-null float64
Population                  4396 non-null float64
Population Group            4396 non-null object
Restaurant Group            4396 non-null object
Income Group                4396 non-null object
dtypes: float64(10), int64(1), object(3)
memory usage: 480.9+ KB


None

### Income groups and heart disease

In [10]:
_ = heart_prev.groupby("Income Group")["Heart Disease Prevalence"].mean().reset_index()
display(_)

,Income Group,Heart Disease Prevalence
0,<50000,6.540580
1,>=50000,4.959305


We saw that the mean prevalence of heart disease is higher in zip codes with a median household income below 50000.

#### Is the observed difference in the samples due to chance?

*  __Null Hypothesis__: There is no difference in the mean heart disease prevalence between groups.
*  __Alternative Hypothesis__: The mean heart disease prevalence is higher when median household income is <50000.

alpha = 0.05

To test the hypotheses we draw bootstrap samples of both groups and compare the bootstrap difference in means to the observed difference in means.

In [11]:
# Functions to draw bootstrap replicates
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func, size=1):
    bs_replicates = np.empty(size)
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data,func)
    return bs_replicates

# Compute the mean of heart disease prevalence
mean_prev = np.mean(heart_prev["Heart Disease Prevalence"])

# Separate heart disease prevalence by low and high income
low_income_prev = heart_prev[heart_prev["Income Group"] == "<50000"]["Heart Disease Prevalence"]
high_income_prev = heart_prev[heart_prev["Income Group"] == ">=50000"]["Heart Disease Prevalence"]

# Calculate the observed difference in means
empirical_diff_means = np.mean(low_income_prev) - np.mean(high_income_prev)

# Generate shifted arrays
low_income_shifted = low_income_prev - np.mean(low_income_prev) + mean_prev
high_income_shifted = high_income_prev - np.mean(high_income_prev) + mean_prev

# Compute 10,000 bootstrap replicates from shifted arrays
bs_replicates_l = draw_bs_reps(low_income_shifted, np.mean, size=10000)
bs_replicates_h = draw_bs_reps(high_income_shifted, np.mean, size=10000)

# Get replicates of difference of means
bs_replicates = bs_replicates_l - bs_replicates_h

# Compute and print p-value: 
p = np.sum(bs_replicates >= empirical_diff_means) / len(bs_replicates)
print('p-value =', p)


p-value = 0.0


A p-value close to 0 indicates we reject the null hypothesis. There is a difference in the means of both groups. The alternative hypothesis suggests that the mean heart disease prevalence is higher when median household income is below 50000 in a zip code.

### Population groups and heart disease

In [12]:
_ = heart_prev.groupby("Population Group")["Heart Disease Prevalence"].mean().reset_index()
display(_)

,Population Group,Heart Disease Prevalence
0,<30000,5.843411
1,>=30000,5.535480


We saw that the mean prevalence of heart disease is higher in zip codes with a population below 10000.

#### Is the observed difference in the samples due to chance?

*  __Null Hypothesis__: There is no difference in the mean heart disease prevalence between groups.
*  __Alternative Hypothesis__: The mean heart disease prevalence is higher when population <10000.

alpha = 0.05

To test the hypotheses we draw bootstrap samples of both groups and compare the bootstrap difference in means to the observed difference in means.

In [13]:
# Functions to draw bootstrap replicates
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func, size=1):
    bs_replicates = np.empty(size)
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data,func)
    return bs_replicates

# Compute the mean of heart disease prevalence
mean_prev = np.mean(heart_prev["Heart Disease Prevalence"])

# Separate heart disease prevalence by population
low_pop_prev = heart_prev[heart_prev["Population Group"] == "<30000"]["Heart Disease Prevalence"]
high_pop_prev = heart_prev[heart_prev["Population Group"] == ">=30000"]["Heart Disease Prevalence"]

# Calculate the observed difference in means
empirical_diff_means = np.mean(low_pop_prev) - np.mean(high_pop_prev)

# Generate shifted arrays
low_pop_shifted = low_pop_prev - np.mean(low_pop_prev) + mean_prev
high_pop_shifted = high_pop_prev - np.mean(high_pop_prev) + mean_prev

# Compute 10,000 bootstrap replicates from shifted arrays
bs_replicates_l = draw_bs_reps(low_pop_shifted, np.mean, size=10000)
bs_replicates_h = draw_bs_reps(high_pop_shifted, np.mean, size=10000)

# Get replicates of difference of means
bs_replicates = bs_replicates_l - bs_replicates_h

# Compute and print p-value: 
p = np.sum(bs_replicates >= empirical_diff_means) / len(bs_replicates)
print('p-value =', p)


p-value = 0.0


A p-value close to 0 indicates we reject the null hypothesis. There is a difference in the means of both groups. The alternative hypothesis suggests that the mean heart disease prevalence is higher when zip code population is below 30000.

#### Summary

The following features have moderate to strong correlations with heart disease prevalence and will be useful for our model:

1. High cholesterol prevalence
2. Hypertension prevalence
3. Diabetes prevalence
4. Sedentarism prevalence
5. Obesity prevalence
6. Smoking prevalence
7. Median household income
8. Population

Restaurant count per zip code may not be helpful for our model. It has a very weak correlation with heart disease prevalence in populations < 30000.